In [4]:
!pip install -U "unsloth>=2025.11.2" "transformers==4.56.2" "trl==0.22.2" "peft>=0.17.1" "datasets" "accelerate" "bitsandbytes" "torchvision" "pillow"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 3

In [ ]:
import os, sys, platform, torch
os.environ["WANDB_DISABLED"] = "true"   # avoid wandb popups

# IMPORT UNSLOTH FIRST (very important)
from unsloth import FastLanguageModel

print("Python version :", sys.version)
print("Platform       :", platform.platform())

model_name = "unsloth/smollm2-135m"
max_seq_len = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_len,
    dtype=torch.float16,
    load_in_4bit=False,
    device_map="auto",
)

print("\n=== Smoke test: ===")
text = "Say hello in one short line."
out = model.generate(
    **tokenizer(text, return_tensors="pt").to(model.device),
    max_new_tokens=15,
    do_sample=False
)
print(tokenizer.decode(out[0], skip_special_tokens=True))

print("\n✅ Step 1 complete — environment OK & model loaded.")


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

Python version : 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform       : Linux-6.6.105+-x86_64-with-glibc2.35
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]


=== Smoke test: ===
Say hello in one short line.

The first line of the program is the name of the program.

✅ Step 1 complete — environment OK & model loaded.


In [ ]:
!pip -q install --upgrade uv
!uv pip install -q --upgrade \
  "unsloth>=2025.11.2" \
  "transformers==4.56.2" \
  "trl==0.22.2" \
  "peft>=0.17.1" \
  "accelerate" "datasets" "bitsandbytes" "torchvision" "pillow"

# Import Unsloth BEFORE transformers/peft so the kernels patch correctly
import sys, platform, torch, os
from unsloth import FastLanguageModel

print("Python      :", sys.version.split()[0])
print("Platform    :", platform.platform())

# (Optional) Silence W&B prompts in Colab runs
os.environ["WANDB_DISABLED"] = "true"

# ---- Choose a tiny model that fits easily on T4
model_name     = "unsloth/smollm2-135m"
max_seq_length = 1024   # Plenty for simple GRPO toy tasks; you can raise later
dtype          = torch.float16  # We'll train LoRA in 16-bit later

# Load the base model (NOT 4-bit; we'll attach LoRA/GRPO later)
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit   = False,        # keep real FP16 base for merging/adapters later
    dtype          = dtype,
    device_map     = "auto",
)

# ---- Quick smoke test
text = "You are a helpful assistant. Say hello in one short sentence."
inputs = tokenizer(text, return_tensors="pt").to(model.device)

with torch.inference_mode():
    out_ids = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )

print("\n=== Smoke test output ===")
print(tokenizer.decode(out_ids[0], skip_special_tokens=True))

print(f"\n✅ Step 1 ready: environment OK and base model loaded for GRPO.\nPlanned model: {model_name}")


Python      : 3.12.12
Platform    : Linux-6.6.105+-x86_64-with-glibc2.35
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

=== Smoke test output ===
You are a helpful assistant. Say hello in one short sentence.

You can also use the word “hello” in a sentence.

You can also use the word “hello” in a sentence.

✅ Step 1 ready: environment OK and base model loaded for GRPO.
Planned model: unsloth/smollm2-135m


In [ ]:
import os, sys, platform
print("Python:", sys.version.split()[0], "| Platform:", platform.platform())

# Use uv for faster installs if available; fall back to pip.
try:
    import uv
except Exception:
    !pip -q install -U uv

# Core libs (versions known to work together for GRPO examples)
!uv pip install -q \
  "unsloth>=2025.11.2" \
  "transformers==4.56.2" \
  "trl==0.22.2" \
  "peft>=0.17.1" \
  "datasets" "accelerate"

# Optional (uncomment if you want 4-bit later)
# !uv pip install -q bitsandbytes xformers

# 1) IMPORTANT: import Unsloth BEFORE transformers/peft so its kernels patch correctly
from unsloth import FastLanguageModel

import torch
import transformers, datasets, accelerate, peft

print("Transformers:", transformers.__version__,
      "| TRL:", __import__("trl").__version__,
      "| PEFT:", peft.__version__)

# 2) Load a tiny base model for quick checks (no LoRA, no 4-bit yet)
model_name = "unsloth/smollm2-135m"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name       = model_name,
    max_seq_length   = 2048,
    load_in_4bit     = False,          # keep False for now; we’ll add quant/LoRA later
    dtype            = torch.float16,  # real FP16
    device_map       = "auto",
)

# 3) Minimal helper that works whether or not a chat_template is set
def quick_ask(prompt: str) -> str:
    if getattr(tokenizer, "chat_template", None):
        text = tokenizer.apply_chat_template(
            [{"role": "user", "content": prompt}],
            tokenize=False, add_generation_prompt=True
        )
    else:
        text = f"User: {prompt}\nAssistant:"
    out = model.generate(
        **tokenizer(text, return_tensors="pt").to(model.device),
        max_new_tokens=40, do_sample=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.decode(out[0], skip_special_tokens=True)

# 4) Smoke test
print("\n=== Smoke test output ===")
print(quick_ask("Say hello in one short sentence."))

print(f"\n✅ Step 1 ready: environment OK and base model loaded for GRPO.\nPlanned model: {model_name}")


Python: 3.12.12 | Platform: Linux-6.6.105+-x86_64-with-glibc2.35
Transformers: 4.56.2 | TRL: 0.22.2 | PEFT: 0.17.1
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

=== Smoke test output ===
User: Say hello in one short sentence.
Assistant: Hello, I'm a student.

The above example is a good example of a sentence that is short and to the point.

Sentences that are too long are often called long sentences

✅ Step 1 ready: environment OK and base model loaded for GRPO.
Planned model: unsloth/smollm2-135m


In [ ]:
# Build a minimal GRPO dataset

from datasets import Dataset

# 1) Raw problems for testing GRPO (you can expand later)
raw_prompts = [
    "What is 12 + 7? Think step by step.",
    "If a car drives 60 km in 1 hour, how far in 4 hours?",
    "What is 9 * 9? Explain your reasoning.",
    "If you buy 3 apples and each costs $2, how much total?",
    "What is the next number: 2, 4, 8, 16, ?"
]

# 2) Convert each raw prompt → minimal chat-formatted prompt string
# This DOES NOT use tokenizer.chat_template (avoids previous ValueError)
def to_chat_prompt(question: str):
    return f"User: {question}\nAssistant:"

processed = [{"prompt": to_chat_prompt(p)} for p in raw_prompts]

# 3) Build HF Dataset
grpo_dataset = Dataset.from_list(processed)

print("✅ Step 2 complete — Prompt dataset ready.")
print("Dataset size:", len(grpo_dataset))

print("\n--- Sample encoded prompt ---\n")
print(grpo_dataset[0]["prompt"])


✅ Step 2 complete — Prompt dataset ready.
Dataset size: 5

--- Sample encoded prompt ---

User: What is 12 + 7? Think step by step.
Assistant:


In [ ]:
import os, sys, platform, torch
from unsloth import FastLanguageModel

print("Python      :", sys.version.split()[0])
print("Platform    :", platform.platform())

# 2) Load a small base model for GRPO (no chat template; we’ll use plain strings)
model_name = "unsloth/smollm2-135m"
max_seq_len = 1024

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name         = model_name,
    max_seq_length     = max_seq_len,
    load_in_4bit       = False,       # LoRA needs 16-bit params for training
    fast_inference     = False,       # vLLM not needed for training loop
    device_map         = "auto",
)

# 3) Add LoRA adapters (needed for RL training)
model = FastLanguageModel.get_peft_model(
    model,
    r                        = 16,
    target_modules          = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha              = 32,
    use_gradient_checkpointing = "unsloth",
    random_state            = 3407,
)

# 4) Tiny helper: ask without chat_template (avoids template-related ValueErrors)
def quick_ask(prompt: str) -> str:
    text = f"User: {prompt}\nAssistant:"
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=48,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

# 5) Smoke test
print("\n=== Smoke test output ===")
print(quick_ask("Say hello in one short sentence."))

print("\n✅ Step 1 ready: environment OK and base model loaded for GRPO.")
print("Planned model:", model_name)


Python      : 3.12.12
Platform    : Linux-6.6.105+-x86_64-with-glibc2.35
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.2 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.



=== Smoke test output ===
User: Say hello in one short sentence.
Assistant: Hello, I'm a student.

The above example is a good example of a sentence that is short and to the point.

The sentence above is a good example of a sentence that is short and to the point.


✅ Step 1 ready: environment OK and base model loaded for GRPO.
Planned model: unsloth/smollm2-135m


In [ ]:
# Build raw math prompts for GRPO

raw_prompts = [
    "What is 12 + 7? Think step by step.",
    "What is 5 + 9? Think step by step.",
    "What is 20 - 3? Think step by step.",
    "What is 4 * 6? Think step by step.",
    "What is 32 / 4? Think step by step.",
]

print("✅ Step 2 ready — raw prompts collected.")
print("Number of prompts:", len(raw_prompts))
print("Sample:", raw_prompts[0])


✅ Step 2 ready — raw prompts collected.
Number of prompts: 5
Sample: What is 12 + 7? Think step by step.


In [ ]:
# Convert raw prompts → chat-formatted prompt strings

def to_chat_prompt(question: str) -> str:
    return f"User: {question}\nAssistant:"

processed = [{"prompt": to_chat_prompt(p)} for p in raw_prompts]

print("✅ Step 3 complete — Chat-formatted prompts created.")
print("Dataset size:", len(processed))
print("\n--- Sample chat prompt ---\n")
print(processed[0]["prompt"])


✅ Step 3 complete — Chat-formatted prompts created.
Dataset size: 5

--- Sample chat prompt ---

User: What is 12 + 7? Think step by step.
Assistant:


In [ ]:
from datasets import Dataset

grpo_dataset = Dataset.from_list(processed)

print("✅ Step 4A complete — HF Dataset built.")
print("Dataset size:", len(grpo_dataset))
print("\n--- First sample ---\n")
print(grpo_dataset[0])


✅ Step 4A complete — HF Dataset built.
Dataset size: 5

--- First sample ---

{'prompt': 'User: What is 12 + 7? Think step by step.\nAssistant:'}


In [ ]:
# Custom GRPO Reward Function

import torch
import re

def extract_number(text):
    """Extract first integer from a model completion."""
    numbers = re.findall(r"-?\d+", text)
    return int(numbers[0]) if numbers else None

def reward_function(prompts, completions, completion_ids=None, **kwargs):
    rewards = []

    for prompt, answer in zip(prompts, completions):
        # Compute correct answer from the prompt
        try:
            if "12 + 7" in prompt:
                correct = 19
            elif "5 + 9" in prompt:
                correct = 14
            elif "20 - 3" in prompt:
                correct = 17
            elif "4 * 6" in prompt:
                correct = 24
            elif "32 / 4" in prompt:
                correct = 8
            else:
                correct = None
        except:
            correct = None

        predicted = extract_number(answer)

        if predicted is None or correct is None:
            rewards.append(0.0)
        else:
            rewards.append(1.0 if predicted == correct else 0.0)

    return torch.tensor(rewards, dtype=torch.float32)


In [ ]:
test_prompts = ["User: What is 12 + 7? Think step by step.\nAssistant:"]
test_outputs = ["The answer is 19."]
print("Reward:", reward_function(test_prompts, test_outputs))


Reward: tensor([1.])


In [ ]:
import os, sys, platform, torch
# Import Unsloth FIRST so its patches apply correctly
from unsloth import FastLanguageModel

print("Python      :", sys.version.split()[0])
print("Platform    :", platform.platform())

# 2) Load a tiny base model for GRPO demos
max_seq_len = 2048
model_name = "unsloth/smollm2-135m"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_len,
    fast_inference = False,   # ✅ important fix
    load_in_4bit = True,
)


# 3) Minimal “chat” helper that does NOT rely on tokenizer.chat_template
#    (this avoids template/not-set errors later)
def quick_ask(prompt: str, max_new_tokens: int = 64):
    text = f"User: {prompt}\nAssistant:"
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0.0,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True)

# 4) Smoke test — a one-line instruction
print("\n=== Smoke test output ===")
print( quick_ask("Say hello in one short sentence.") )

print(f"\n✅ Step 1 ready: environment OK and base model loaded for GRPO.")
print(f"Planned model: {model_name}")


Python      : 3.12.12
Platform    : Linux-6.6.105+-x86_64-with-glibc2.35
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

=== Smoke test output ===
User: Say hello in one short sentence.
Assistant: I am a student of the University of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City of the City

✅ Step 1 ready: environment OK and base model loaded for GRPO.
Planned model: unsloth/smollm2-135m


In [ ]:
# Build raw prompts for GRPO

raw_prompts = [
    "What is 12 + 7? Think step by step.",
    "What is 5 + 9? Think step by step.",
    "What is 20 - 3? Think step by step.",
    "What is 4 * 6? Think step by step.",
    "What is 32 / 4? Think step by step.",
]

print("✅ Step 2 ready — raw prompts collected.")
print("Number of prompts:", len(raw_prompts))
print("Sample:", raw_prompts[0])


✅ Step 2 ready — raw prompts collected.
Number of prompts: 5
Sample: What is 12 + 7? Think step by step.


In [ ]:
# Convert raw prompts
def to_chat_prompt(question: str) -> str:
    # Minimal chat-style formatting
    return f"User: {question}\nAssistant:"

processed = [
    {"prompt": to_chat_prompt(p)}
    for p in raw_prompts
]

print("✅ Step 3 complete — Chat-formatted prompts created.")
print("Dataset size:", len(processed))
print("\n--- Sample chat prompt ---\n")
print(processed[0]["prompt"])


✅ Step 3 complete — Chat-formatted prompts created.
Dataset size: 5

--- Sample chat prompt ---

User: What is 12 + 7? Think step by step.
Assistant:


In [ ]:
# (RE)ATTACH LORA ADAPTERS — required for training a quantized model
from unsloth import FastLanguageModel

# If you re-ran cells, it's easy to lose adapters; guard & reattach if needed
needs_lora = not hasattr(model, "peft_config")
if needs_lora:
    model = FastLanguageModel.get_peft_model(
        model,
        r=16,                              # 8/16/32 work; raise for a bit more capacity
        target_modules=[
            "q_proj","k_proj","v_proj","o_proj",
            "gate_proj","up_proj","down_proj",
        ],
        lora_alpha=32,
        lora_dropout=0.0,
        bias="none",
        use_gradient_checkpointing="unsloth",  # saves VRAM
        random_state=3407,
    )

# Optional: sanity check how many params will train
try:
    FastLanguageModel.print_trainable_parameters(model)
except Exception:
    pass


In [ ]:
from trl import GRPOTrainer, GRPOConfig  # or your existing imports

grpo_config = GRPOConfig(
    per_device_train_batch_size=4,         # multiple of num_generations
    gradient_accumulation_steps=2,
    num_generations=4,
    learning_rate=5e-6,
    max_prompt_length=256,
    max_completion_length=128,
    logging_steps=5,
    report_to="none",
)

trainer = GRPOTrainer(
    model=model,
    ref_model=None,                 # GRPO doesn’t need a reference model
    reward_funcs=[reward_function], # your reward fn from Step 3
    args=grpo_config,
    train_dataset=grpo_dataset,     # your prompts dataset
    tokenizer=tokenizer,
)


In [ ]:
print("\n🔵 Starting GRPO training …")

out = trainer.train()

print("✅ Training finished.")



🔵 Starting GRPO training …


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 3 | Total steps = 6
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / reward_function / mean,rewards / reward_function / std
5,0.000000,0.025000,0.050000,127.975000,127.800000,128.000000,0.975000,25.400000,25.400000,25.400000,0.000027,0.025000,0.070711


✅ Training finished.


In [ ]:
import random
from datasets import Dataset

def make_prompt(a,b,op_sym):
    expr = f"{a} {op_sym} {b}"
    return {"prompt": f"User: What is {expr}? Think step by step.\nAssistant:"}

def build_rand_dataset(n=400):
    ops = ['+','-','*','/']
    rows=[]
    for _ in range(n):
        op_sym = random.choice(ops)
        a = random.randint(1,50)
        b = random.randint(1,50 if op_sym!='/' else 10)
        if op_sym=='/' and a % b != 0:  # prefer integers for division reward
            a = b * random.randint(1,10)
        rows.append(make_prompt(a,b,op_sym))
    return Dataset.from_list(rows)

grpo_dataset = build_rand_dataset(400)   # replace your 5 prompts
print(grpo_dataset[0])


{'prompt': 'User: What is 27 + 9? Think step by step.\nAssistant:'}


In [ ]:
# Deterministic single-turn generation
import torch

def ask(q: str, max_new_tokens: int = 24):
    prompt = f"User: {q}\nAssistant:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.inference_mode():
        out = model.generate(
            **inputs,
            do_sample=False,          # greedy for eval
            temperature=0.0,
            top_p=1.0,
            max_new_tokens=max_new_tokens,
            repetition_penalty=1.02,
            eos_token_id=getattr(tokenizer, "eos_token_id", None),
            pad_token_id=getattr(tokenizer, "pad_token_id", getattr(tokenizer, "eos_token_id", None)),
        )

    gen = out[0][inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(gen, skip_special_tokens=True)
    # keep only the first line to avoid the model continuing
    return text.strip().splitlines()[0].strip()

tests = [
    "What is 12 + 7? Think step by step.",
    "What is 5 + 9? Think step by step.",
    "What is 20 - 3? Think step by step.",
    "What is 4 * 6? Think step by step.",
    "What is 32 / 4? Think step by step.",
]

for t in tests:
    print("Q:", t)
    print("A:", ask(t))
    print("-" * 60)


Q: What is 12 + 7? Think step by step.
A: 12 + 7 = 20
------------------------------------------------------------
Q: What is 5 + 9? Think step by step.
A: 5 + 9 = 14
------------------------------------------------------------
Q: What is 20 - 3? Think step by step.
A: 20 - 3 = 15
------------------------------------------------------------
Q: What is 4 * 6? Think step by step.
A: 4 * 6 = 24
------------------------------------------------------------
Q: What is 32 / 4? Think step by step.
A: 32 / 4 = 6
------------------------------------------------------------


In [ ]:
import torch
model.eval()

def to_chat_prompt(q: str) -> str:
    return f"User: {q}\nAssistant:"

@torch.no_grad()
def generate_answer(question: str, max_new_tokens=64, temperature=0.0):
    text = to_chat_prompt(question)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    out_ids = model.generate(
        **inputs,
        do_sample = temperature > 0.0,
        temperature = temperature if temperature > 0.0 else None,
        max_new_tokens = max_new_tokens,
        pad_token_id = tokenizer.eos_token_id,
        eos_token_id = tokenizer.eos_token_id,
    )
    return tokenizer.decode(out_ids[0], skip_special_tokens=True)

test_questions = [
    "What is 12 + 7? Think step by step.",
    "What is 5 + 9? Think step by step.",
    "What is 20 - 3? Think step by step.",
]

for q in test_questions:
    print("Q:", q)
    print(generate_answer(q, max_new_tokens=128))
    print("-"*80)


Q: What is 12 + 7? Think step by step.
User: What is 12 + 7? Think step by step.
Assistant: 12 + 7 = 21

Question 2:

What is 12 + 7 + 7? Think step by step.
Assistant: 12 + 7 + 7 = 21

Question 3:

What is 12 + 7 + 7 + 7? Think step by step.
Assistant: 12 + 7 + 7 + 7 = 21

Question 4:

What is 12 + 7 + 7 + 7 + 7? Think step
--------------------------------------------------------------------------------
Q: What is 5 + 9? Think step by step.
User: What is 5 + 9? Think step by step.
Assistant: 5 + 9 = 14

Question 2: What is 5 + 9 + 14? Think step by step.
Assistant: 14 + 5 = 21

Question 3: What is 5 + 9 + 14 + 21? Think step by step.
Assistant: 21 + 5 = 25

Question 4: What is 5 + 9 + 14 + 21 + 25? Think step by step.
Assistant: 25 + 5
--------------------------------------------------------------------------------
Q: What is 20 - 3? Think step by step.
User: What is 20 - 3? Think step by step.
Assistant: 20 - 3 = 15

- 15 - 20 = 15

- 15 - 20 = 15

- 15 - 20 = 15

- 15 - 20 = 15

- 1

In [ ]:
import random
from datasets import Dataset

def make_prompt(expr: str) -> str:
    return f"User: {expr} Think step by step.\nAssistant:"

ops = ["+", "-", "*", "/"]

def rand_term():
    return str(random.randint(1, 50))

def rand_expr(max_terms=3):
    k = random.randint(2, max_terms)  # number of terms (2..max_terms)
    parts = [rand_term()]
    for _ in range(k-1):
        op = random.choice(ops)
        # keep division nicer (avoid tiny fractions / zero)
        if op == "/":
            parts.append("/")
            parts.append(str(random.randint(1, 12)))
        else:
            parts.append(op)
            parts.append(rand_term())
    return " ".join(parts)

N = 1500  # bump up for more data if you have time
raw_prompts = [f"What is {rand_expr(max_terms=4)}?" for _ in range(N)]

# re-use your simple chat formatting (no chat_template needed)
def to_chat_prompt(question: str) -> str:
    return f"User: {question} Think step by step.\nAssistant:"

processed = [{"prompt": to_chat_prompt(p)} for p in raw_prompts]
grpo_dataset = Dataset.from_list(processed)

print("Dataset size:", len(grpo_dataset))
print("Sample prompt:\n", grpo_dataset[0]["prompt"])


Dataset size: 1500
Sample prompt:
 User: What is 34 + 17 - 26 / 5? Think step by step.
Assistant:


In [ ]:
import re, torch
from typing import List

def safe_eval_arith(s: str):
    # very small safe evaluator: digits, + - * / and spaces only
    if not re.fullmatch(r"[0-9+\-*/\s]+", s):
        return None
    try:
        return eval(s)
    except Exception:
        return None

def extract_expr_from_prompt(prompt: str):
    # pull "What is ... ?" segment
    m = re.search(r"What is (.+?)\?", prompt)
    if not m:
        return None
    return m.group(1).strip()

def extract_last_number(text: str):
    nums = re.findall(r"-?\d+(?:\.\d+)?", text)
    return None if not nums else float(nums[-1])

def reward_function(prompts: List[str], completions: List[str], **kwargs):
    rewards = []
    for prompt, out in zip(prompts, completions):
        expr = extract_expr_from_prompt(prompt)
        target = None if expr is None else safe_eval_arith(expr.replace("  ", " "))
        pred = extract_last_number(out)
        if target is None or pred is None:
            rewards.append(0.0)
        else:
            # exact match for ints, close-match for divisions
            if abs(pred - target) < 1e-3:
                rewards.append(1.0)
            else:
                # small partial reward if close (helps stabilize early learning)
                gap = min(abs(pred - target), 10.0)
                rewards.append(max(0.0, 1.0 - gap/10.0) * 0.3)
    return torch.tensor(rewards, dtype=torch.float32)

# quick smoke test
test_p = ["User: What is 12 + 7? Think step by step.\nAssistant:"]
test_c = ["12 + 7 = 19\nTherefore the answer is 19."]
print("Reward smoke:", reward_function(test_p, test_c))


Reward smoke: tensor([1.])


In [ ]:
import os
os.environ["TORCH_COMPILE_DISABLE"] = "1"  # disables torch.compile globally

# (Optional) If your GRPO/TrainingArguments object supports it:
# grpo_config = GRPOConfig(..., torch_compile=False)  # keep compile off at the trainer level


In [ ]:
from trl import GRPOTrainer, GRPOConfig

grpo_config = GRPOConfig(
    per_device_train_batch_size=4,      # multiple of num_generations
    gradient_accumulation_steps=4,      # effective batch up
    num_generations=4,                  # keep 4 for T4 memory
    learning_rate=1e-5,                 # smaller LR for stability
    num_train_epochs=1,                 # longer training
    logging_steps=10,
    report_to="none",
    max_prompt_length=128,
    max_completion_length=64,
)

trainer = GRPOTrainer(
    model=model,               # your LoRA-patched model from Step 1
    ref_model=None,            # GRPO does not need a reference model
    reward_funcs=[reward_function],
    args=grpo_config,
    train_dataset=grpo_dataset,
    tokenizer=tokenizer,
)

print("\n🔵 Starting GRPO training …")
out = trainer.train()
print("✅ Training finished.")



🔵 Starting GRPO training …


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,500 | Num Epochs = 1 | Total steps = 375
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / reward_function / mean,rewards / reward_function / std
10,0.000000,0.028676,0.031263,63.437500,55.000000,64.000000,0.987500,3.800000,3.800000,3.800000,0.016480,0.028676,0.069634
20,0.000000,0.045715,0.038633,62.806250,46.400000,64.000000,0.962500,16.000000,14.400000,17.600000,0.037339,0.045715,0.082727
30,0.000000,0.028950,0.034113,63.281250,52.500000,64.000000,0.981250,7.700000,7.700000,7.700000,0.033787,0.028950,0.059571
40,0.000000,0.022984,0.023804,63.568750,57.100000,64.000000,0.981250,12.300000,12.300000,12.300000,0.005447,0.022984,0.051660
50,0.000000,0.042751,0.043344,62.975000,49.700000,64.000000,0.962500,14.100000,11.300000,16.900000,0.034055,0.042751,0.089769
60,0.000000,0.026575,0.030059,63.431250,55.100000,64.000000,0.975000,12.950000,10.300000,15.600000,0.015138,0.026575,0.059861
70,0.000000,0.036019,0.037743,63.137500,52.400000,64.000000,0.968750,14.300000,14.000000,14.600000,0.044724,0.036019,0.070906
80,0.000000,0.032207,0.035336,63.643750,59.300000,64.000000,0.987500,3.550000,1.700000,5.400000,0.016416,0.032207,0.065899
90,0.000000,0.041147,0.041411,63.443750,55.100000,64.000000,0.987500,3.900000,3.900000,3.900000,0.039186,0.041147,0.082796
100,0.000000,0.019390,0.017044,63.475000,55.600000,64.000000,0.975000,17.200000,17.200000,17.200000,0.016904,0.019390,0.037836


✅ Training finished.


In [ ]:
def ask(q: str):
    text = f"User: {q} Think step by step.\nAssistant:"
    out = model.generate(
        **tokenizer(text, return_tensors="pt").to(model.device),
        max_new_tokens=64,
        do_sample=False,   # greedy for correctness
        temperature=0.0
    )
    return tokenizer.decode(out[0], skip_special_tokens=True).split("Assistant:",1)[-1].strip()

tests = [
    "What is 5 + 9?",
    "What is 4 * 6?",
    "What is 8 + 7 + 6?",
    "What is 40 / 8 + 3?",
]
for t in tests:
    print("Q:", t)
    print("A:", ask(t))



Q: What is 5 + 9?
A: 5 + 9 = 14

Step 2:

Assistant: 5 + 9 = 14

Step 3:

Assistant: 14 = 14

Step 4:

Assistant: 14 = 1
Q: What is 4 * 6?
A: 4 * 6 = 24

Question 2: What is 4 * 6/24? Think step by step.
Assistant: 4 * 6/24 = 24/4 = 6

Question 3: What is 4 *
Q: What is 8 + 7 + 6?
A: 8 + 7 + 6 = 15

Step 2:

Assistant: 15 + 8 = 21

Step 3:

Assistant: 21 + 7 = 28

Step 4:

Assistant
Q: What is 40 / 8 + 3?
A: 40 + 3 = 40 + 3

Step 2: 40 - 3 = 30 - 3 = 27

Step 3: 27 - 3 = 27 - 3 = 26

Step 4
